Week 1

In [26]:
!pip install pandas numpy matplotlib seaborn scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [27]:
import pandas as pd 
import numpy as np

from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [28]:
df = pd.read_csv('afa2e701598d20110228.csv', sep=';')
print("Original data loaded. Shape:", df.shape)
df

Original data loaded. Shape: (2861, 11)


,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL
0,1,17.02.2000,0.330,2.77,12.0,12.30,9.50,0.057,154.00,0.454,289.50
1,1,11.05.2000,0.044,3.00,51.6,14.61,17.75,0.034,352.00,0.090,1792.00
2,1,11.09.2000,0.032,2.10,24.5,9.87,13.80,0.173,416.00,0.200,2509.00
3,1,13.12.2000,0.170,2.23,35.6,12.40,17.13,0.099,275.20,0.377,1264.00
4,1,02.03.2001,0.000,3.03,48.8,14.69,10.00,0.065,281.60,0.134,1462.00
...,...,...,...,...,...,...,...,...,...,...,...
2856,22,06.10.2020,0.046,2.69,3.6,8.28,3.80,0.038,160.00,0.726,77.85
2857,22,27.10.2020,0.000,1.52,0.5,11.26,0.56,0.031,147.20,0.634,71.95
2858,22,03.12.2020,0.034,0.29,0.8,11.09,2.58,0.042,209.92,0.484,61.17
2859,22,12.01.2021,0.000,2.10,0.0,14.31,3.94,0.034,121.60,0.424,63.49


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2861 entries, 0 to 2860
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         2861 non-null   int64  
 1   date       2861 non-null   object 
 2   NH4        2858 non-null   float64
 3   BSK5       2860 non-null   float64
 4   Suspended  2845 non-null   float64
 5   O2         2858 non-null   float64
 6   NO3        2860 non-null   float64
 7   NO2        2858 non-null   float64
 8   SO4        2812 non-null   float64
 9   PO4        2833 non-null   float64
 10  CL         2812 non-null   float64
dtypes: float64(9), int64(1), object(1)
memory usage: 246.0+ KB


In [30]:
df['date'] = pd.to_datetime(df['date'])

C:\Users\Hp\AppData\Local\Temp\ipykernel_19812\3532345252.py:1: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['date'] = pd.to_datetime(df['date'])


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2861 entries, 0 to 2860
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         2861 non-null   int64         
 1   date       2861 non-null   datetime64[ns]
 2   NH4        2858 non-null   float64       
 3   BSK5       2860 non-null   float64       
 4   Suspended  2845 non-null   float64       
 5   O2         2858 non-null   float64       
 6   NO3        2860 non-null   float64       
 7   NO2        2858 non-null   float64       
 8   SO4        2812 non-null   float64       
 9   PO4        2833 non-null   float64       
 10  CL         2812 non-null   float64       
dtypes: datetime64[ns](1), float64(9), int64(1)
memory usage: 246.0 KB


In [32]:
df = df.sort_values(by=['id', 'date'])

In [33]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

In [34]:
pollutants = ['O2', 'NO3', 'NO2', 'SO4', 'PO4', 'CL']

In [35]:
df = df.dropna(subset=pollutants)
df.head()

,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL,year,month
0,1,2000-02-17,0.330,2.77,12.0,12.30,9.50,0.057,154.0,0.454,289.5,2000,2
1,1,2000-05-11,0.044,3.00,51.6,14.61,17.75,0.034,352.0,0.090,1792.0,2000,5
2,1,2000-09-11,0.032,2.10,24.5,9.87,13.80,0.173,416.0,0.200,2509.0,2000,9
3,1,2000-12-13,0.170,2.23,35.6,12.40,17.13,0.099,275.2,0.377,1264.0,2000,12
4,1,2001-03-02,0.000,3.03,48.8,14.69,10.00,0.065,281.6,0.134,1462.0,2001,3


In [36]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
id,2780.0,12.279856,1.0,8.0,13.0,16.0,22.0,6.072261
date,2780,2010-04-11 19:10:57.841726464,2000-01-10 00:00:00,2005-03-17 00:00:00,2010-05-12 00:00:00,2015-06-09 00:00:00,2021-04-07 00:00:00,NaN
NH4,2778.0,0.634145,0.0,0.08,0.22,0.48,30.26,1.829515
BSK5,2780.0,4.248248,0.0,2.12,3.745,5.7,27.12,2.760254
Suspended,2778.0,12.881451,0.0,6.0,10.0,14.4,595.0,16.655984
O2,2780.0,9.549462,0.0,7.1,9.0,11.6325,90.0,4.453361
NO3,2780.0,4.349107,0.0,1.4,2.825,5.6,133.4,6.953503
NO2,2780.0,0.224384,0.0,0.03,0.0575,0.12,109.0,2.157737
SO4,2780.0,59.671929,0.003,27.1,37.8,65.145,3573.4,97.074655
PO4,2780.0,0.393275,0.0,0.12975,0.27,0.46,11.66,0.640663


In [37]:
print(f"Shape after dropping rows with missing values: {df.shape}")

Shape after dropping rows with missing values: (2780, 13)


Week 2

In [38]:
X = df[['id', 'year','month']]
y = df[pollutants]

In [39]:
X_encoded = pd.get_dummies(X, columns=['id'], drop_first=True)

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.18, random_state=43)

In [46]:
model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=43))
model.fit(X_train, y_train)

MultiOutputRegressor(estimator=RandomForestRegressor(random_state=43))

In [47]:
y_pred = model.predict(X_test) # predict the output

In [48]:
print("Model Performance on the Test Data:")
for i, pollutant in enumerate(pollutants):
    print(f'Pollutant: {pollutant}')
    print('MSE:', mean_squared_error(y_test.iloc[:, i], y_pred[:, i]))
    print('R2:', r2_score(y_test.iloc[:, i], y_pred[:, i]))
    print()

Model Performance on the Test Data:
Pollutant: O2
MSE: 18.243931826060717
R2: -0.5466092975906944

Pollutant: NO3
MSE: 115.4863341469193
R2: -1.613713620807769

Pollutant: NO2
MSE: 26.250649859174978
R2: -65.03801414273211

Pollutant: SO4
MSE: 23920.45803936937
R2: -3.9990374583896005

Pollutant: PO4
MSE: 0.23216160446554573
R2: 0.19931439353389457

Pollutant: CL
MSE: 310195.0795197903
R2: -1.560194441725212



In [49]:
station_id = '22'
year_input = 2024
month_input = 6
input_data = pd.DataFrame({'year': [year_input], 'month':[month_input], 'id': [station_id]})
input_encoded = pd.get_dummies(input_data, columns=['id'])

# Align with training feature columns
missing_cols = set(X_encoded.columns) - set(input_encoded.columns)
for col in missing_cols:
    input_encoded[col] = 0
input_encoded = input_encoded[X_encoded.columns]  # reorder columns

# Predict pollutants
predicted_pollutants = model.predict(input_encoded)[0]

print(f"\nPredicted pollutant levels for station '{station_id}' in {year_input}-{month_input}:")
for p, val in zip(pollutants, predicted_pollutants):
    print(f"  {p}: {val:.2f}")


Predicted pollutant levels for station '22' in 2024-6:
  O2: 12.41
  NO3: 5.68
  NO2: 0.03
  SO4: 128.98
  PO4: 0.36
  CL: 64.04


In [50]:
import joblib

joblib.dump(model, 'prediction_model.pkl')
joblib.dump(X_encoded.columns.tolist(), "model_columns.pkl")
print('Model and columns structure are saved!')

Model and columns structure are saved!
